In [1]:
import itertools

import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx

import meshplot as mp
import pyvista as pv
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from src import shapes

# Define The Figure and the Morse Function

In [2]:
def get_linear_morse(vector=None):
    if vector is None:
        vector = np.random.random(4)
    vector = np.array(vector)
    def f(points):
        return points @ vector
    return f

direction = np.random.random(3)
direction /= np.linalg.norm(direction)

In [3]:
f = lambda p: np.linalg.norm(p, axis=-1, ord=2)
#f = lambda p: p[:, -1]
#f = lambda p: (np.random.random(3)*p).sum(axis=-1)*(np.random.random(3)*p).sum(axis=-1)

In [4]:
def cylindrical_twist(vertices, k=-0.3, mode="x", scale=1.5):
    """
    Nonlinear cylindrical twist diffeomorphism on R^3.

    vertices: (n,3) array
    mode:
      - "z": angle depends on z  (theta = k * tanh(z/scale))
      - "r": angle depends on radius r (theta = k * tanh(r/scale))
    k: twist strength (radians, roughly bounded by +/-k for tanh)
    scale: controls how quickly tanh saturates
    """
    if mode == 'x':
        v = vertices[:, [1, 2, 0]]
        v = cylindrical_twist(v, k=k, mode="z", scale=scale)
        v = v[:, [2, 0, 1]]
        return v
        
        
    v = vertices.copy()
    x, y, z = v[:, 0], v[:, 1], v[:, 2]

    r = np.sqrt(x*x + y*y)

    if mode == "z":
        theta = k * np.tanh(z / scale)
    elif mode == "r":
        theta = k * np.tanh(r / scale)
    else:
        raise ValueError('mode must be "z" or "r"')

    c, s = np.cos(theta), np.sin(theta)

    v[:, 0] = c * x - s * y
    v[:, 1] = s * x + c * y
    
    return v

In [11]:
n, m = 13, 12
vertices, faces = shapes.get_halftori_bouquet(leaves=3, n=n, m=m, l0=1.0, glue=False)

vertices = cylindrical_twist(vertices)

vertices, faces = shapes.split_large_edges(vertices, faces, max_length=1.0)


print(f'faces.shape = {faces.shape}')

face_mean_values = f(vertices[faces]).mean(axis=1)

p = mp.plot(vertices, faces, face_mean_values, shading={"wireframe": True})

faces.shape = (1352, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.72…

In [12]:
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3, dtype=faces.dtype), faces]).ravel()

mesh = pv.PolyData(vertices, faces_pv)
mesh.point_data["values"] = f(vertices)  # per-vertex scalars

p = pv.Plotter(window_size=(600, 600))
p.add_mesh(
    mesh,
    scalars="values",
    cmap="viridis",
    smooth_shading=False,   # helps show linear interpolation nicely
    show_edges=True,      # set True if you want to see triangle edges
)
p.add_scalar_bar(title="values")
p.show()

Widget(value='<iframe src="http://localhost:35059/index.html?ui=P_0x70bb18127d40_2&reconnect=auto" class="pyvi…

# Paths

In [13]:
from src.ms import MorseSmale

In [14]:
ms = MorseSmale(faces, f(vertices), vertices)

paths = list(ms.iterate_paths())

In [15]:
def plot_paths(ms: MorseSmale, 
               min_color="lime", saddle_color="pink", max_color='orangered', point_size=12, 
               value_cmap="viridis", path_cmap="rainbow", line_width=4, 
               window_size=(1000, 500), eps=0.05, opacity0=1.0, opacity1=0.6):
    """
    """
    path_lines = [ms.vertices[chain] for chain in ms.iterate_paths()]
    path_lines_shifted = []
    path_adds = np.zeros(ms.n_vertices)
    for path in ms.iterate_paths():
        path_adds[ms.mins + ms.maxs + ms.saddles] = 0
        path_lines_shifted.append(vertices[path] + eps*path_adds[path].reshape(-1, 1))
        path_adds[path] += 1
    path_colors = [mcolors.to_hex(c) for c in plt.get_cmap(path_cmap)(np.linspace(0, 1, len(path_lines)))]
    np.random.shuffle(path_colors)

    faces_pv = np.hstack([np.full((ms.faces.shape[0], 1), 3, dtype=faces.dtype), ms.faces]).ravel()
    mesh = pv.PolyData(ms.vertices, faces_pv)
    mesh.point_data["values"] = ms.values
    
    pl = pv.Plotter(shape=(1, 2), window_size=window_size)

    pl.subplot(0, 0)
    pl.add_mesh(mesh, scalars="values", cmap=value_cmap, smooth_shading=False, show_edges=True, opacity=opacity0)
    pl.add_points(ms.vertices[ms.mins], color=min_color, point_size=point_size, render_points_as_spheres=True)
    pl.add_points(ms.vertices[ms.saddles], color=saddle_color, point_size=point_size, render_points_as_spheres=True)
    pl.add_points(ms.vertices[ms.maxs], color=max_color, point_size=point_size, render_points_as_spheres=True)
    for line in path_lines:
        pl.add_mesh(pv.lines_from_points(line), color="white", line_width=line_width)

    
    pl.subplot(0, 1)
    pl.add_mesh(mesh, color='white', smooth_shading=False, show_edges=True, opacity=opacity1)
    pl.add_points(ms.vertices[ms.mins], color=min_color, point_size=point_size, render_points_as_spheres=True)
    pl.add_points(ms.vertices[ms.saddles], color=saddle_color, point_size=point_size, render_points_as_spheres=True)
    pl.add_points(ms.vertices[ms.maxs], color=max_color, point_size=point_size, render_points_as_spheres=True)
    for i, (line, color) in enumerate(zip(path_lines_shifted, path_colors)):
        pl.add_mesh(pv.lines_from_points(line), color=color, line_width=line_width)

    
    pass
    pl.link_views()
    return pl

In [16]:
pl = plot_paths(ms)

pl.show()

Widget(value='<iframe src="http://localhost:35059/index.html?ui=P_0x70bb309fb4a0_3&reconnect=auto" class="pyvi…